In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

from sklearn.inspection import permutation_importance

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
import logging as log
from sklearn.metrics import log_loss, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA

from sklearn import config_context

import os

import circlify
import plotly.express as px

import xgboost as xgb

import shap

In [3]:
bankA_data = pd.read_csv("BankA_Clean.csv")
bankB_data = pd.read_csv("BankB_Clean.csv")
bankC_data = pd.read_csv("BankC_Clean.csv")

In [36]:
bank_files = {"A": "BankA_Clean.csv", "B": "BankB_Clean.csv", "C": "BankC_Clean.csv"}
banks = {key: pd.read_csv(file) for key, file in bank_files.items()}

drop_features = ["race_Black", "sex_Male"]

education_low = ["education_Preschool"]
education_medium = ["education_HS-grad", "education_Some-college", "education_Assoc-acdm", "education_Assoc-voc"]
education_high = ["education_Bachelors", "education_Masters", "education_Doctorate", "education_Prof-school"]

for key, df in banks.items():
    df["income"] = df["income"].astype(int)
    df.drop(columns=drop_features, errors="ignore", inplace=True)

    df["education_group"] = "Low"
    df.loc[df[education_medium].sum(axis=1) > 0, "education_group"] = "Medium"
    df.loc[df[education_high].sum(axis=1) > 0, "education_group"] = "High"
    df.drop(columns=education_low + education_medium + education_high, errors="ignore", inplace=True)

    df["marital_status_group"] = "Single"
    df.loc[df.get("marital-status_Married-civ-spouse", 0) == 1, "marital_status_group"] = "Married"
    df.drop(columns=[col for col in df.columns if "marital-status" in col], errors="ignore", inplace=True)

    occupation_mid = ["occupation_Exec-managerial", "occupation_Tech-support"]
    occupation_high = ["occupation_Prof-specialty"]
    df["occupation_group"] = "Low-Skill"
    df.loc[df[occupation_mid].sum(axis=1) > 0, "occupation_group"] = "Mid-Skill"
    df.loc[df[occupation_high].sum(axis=1) > 0, "occupation_group"] = "High-Skill"

    relationship_mapping = {
        "relationship_Own-child": "dependent",
        "relationship_Wife": "married",
        "relationship_Not-in-family": "independent",
        "relationship_Unmarried": "single",
        "relationship_Other-relative": "dependent"
    }
    df["relationship_group"] = df.apply(lambda row: next((v for k, v in relationship_mapping.items() if row.get(k, 0) == 1), "unknown"), axis=1)

    workclass_mapping = {
        "workclass_Private": "private",
        "workclass_Self-emp-not-inc": "self-employed",
        "workclass_Local-gov": "government",
        "workclass_State-gov": "government"
    }
    df["workclass_group"] = df.apply(lambda row: next((v for k, v in workclass_mapping.items() if row.get(k, 0) == 1), "other"), axis=1)

    categorical_columns = ["education_group", "marital_status_group", "occupation_group", "relationship_group", "workclass_group"]
    df[categorical_columns] = df[categorical_columns].astype("category")

    df.to_csv(f"bank{key}_final.csv", index=False)

print("Data cleansing done, saving files as 'bankA_final.csv', 'bankB_final.csv', 'bankC_final.csv'.")


Data cleansing done, saving files as 'bankA_final.csv', 'bankB_final.csv', 'bankC_final.csv'.
